In [1]:
val bootstrap_servers = "kafka05-prod01.messagehub.services.us-south.bluemix.net:9093,kafka03-prod01.messagehub.services.us-south.bluemix.net:9093,kafka04-prod01.messagehub.services.us-south.bluemix.net:9093,kafka01-prod01.messagehub.services.us-south.bluemix.net:9093,kafka02-prod01.messagehub.services.us-south.bluemix.net:9093"
val username = "changeme"
val password = "changeme"
val topic    = "mytopic"

In [2]:
import net.christophersnow.config.MessageHubConfig
import net.christophersnow.dstream.KafkaStreaming.KafkaStreamingContextAdapter

import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf

import org.apache.spark.streaming.Duration
import org.apache.spark.streaming.Seconds
import org.apache.spark.streaming.StreamingContext

import org.apache.kafka.common.serialization.Deserializer
import org.apache.kafka.common.serialization.StringDeserializer

val kafkaProps = new MessageHubConfig

kafkaProps.setConfig("bootstrap.servers",   bootstrap_servers)
kafkaProps.setConfig("kafka.user.name",     username)
kafkaProps.setConfig("kafka.user.password", password)
kafkaProps.setConfig("kafka.topic",         topic)

kafkaProps.createConfiguration()

val ssc = new StreamingContext( sc, Seconds(60) )

val stream = ssc.createKafkaStream[String, String, StringDeserializer, StringDeserializer](
  kafkaProps,
  List(kafkaProps.getConfig("kafka.topic"))
  );

stream.foreachRDD{ rdd =>
  // we only want to create a folder in hdfs if we have some data
  if (rdd.count() > 0) {
    def uuid = java.util.UUID.randomUUID.toString
    val outDir = s"test-${uuid}"
    rdd.saveAsTextFile (outDir)
  }
}

stream.print()
ssc.start()
ssc.awaitTermination()



default location of ssl Trust store is: /usr/local/src/spark160master/ibm-java-x86_64-80/jre/lib/security/cacerts
Registering JaasConfiguration: /gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/notebook/tmp/bpyCprMeLfavhUmQ/jaas.conf
default location of ssl Trust store is: /usr/local/src/spark160master/ibm-java-x86_64-80/jre/lib/security/cacerts
-------------------------------------------
Time: 1493061180000 ms
-------------------------------------------

-------------------------------------------
Time: 1493061240000 ms
-------------------------------------------
(null,123)

